## Import Required Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import psycopg2

## Load Dataset

In [2]:
conn = psycopg2.connect(
    host = "localhost",
    database = "internprojects",
    password = "2526",
    user = "postgres",
    port = "5432"
)

In [3]:
cur = conn.cursor()


# Table names fetch
cur.execute("""
    SELECT table_name 
    FROM information_schema.tables
    WHERE table_schema = 'public';
""")

tables = cur.fetchall()

# Print table names
print("Tables in database:")
for table in tables:
    print(table[0])

Tables in database:
netflix1
supermart_grocery_sales


In [4]:
data = pd.read_sql('SELECT * FROM "netflix1";',conn)
data.head()

,show_id,type,title,director,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,9/25/2021,2020,PG-13,90 min,Documentaries
1,s3,TV Show,Ganglands,Julien Leclercq,France,9/24/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
2,s6,TV Show,Midnight Mass,Mike Flanagan,United States,9/24/2021,2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries"
3,s14,Movie,Confessions of an Invisible Girl,Bruno Garotti,Brazil,9/22/2021,2021,TV-PG,91 min,"Children & Family Movies, Comedies"
4,s8,Movie,Sankofa,Haile Gerima,United States,9/24/2021,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies"


## Data Cleaning

In [5]:
df = data.copy()

In [6]:
# Check Columns, data types and Missing Values.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8790 entries, 0 to 8789
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8790 non-null   object
 1   type          8790 non-null   object
 2   title         8790 non-null   object
 3   director      8790 non-null   object
 4   country       8790 non-null   object
 5   date_added    8790 non-null   object
 6   release_year  8790 non-null   int64 
 7   rating        8790 non-null   object
 8   duration      8790 non-null   object
 9   listed_in     8790 non-null   object
dtypes: int64(1), object(9)
memory usage: 686.8+ KB


In [7]:
# Check for missing values in each columns.
df.isnull().sum()

show_id         0
type            0
title           0
director        0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
dtype: int64

In [8]:
# Convert 'date_added' to datetime.
df['date_added'] = pd.to_datetime(
    data['date_added'], 
    format='mixed', 
    dayfirst=True, 
    errors='coerce'
)

In [9]:
# Remove extra spaces from string columns.
for col in ['type', 'title', 'director', 'country', 'rating', 'duration', 'listed_in']:
    df[col] = df[col].str.strip()

In [10]:
# Drop duplicates if any
df.drop_duplicates(inplace=True)

In [11]:
# Date-Based Features
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month
df['month_name'] = df['date_added'].dt.month_name()

In [12]:
# Duration Features
df['duration_mins'] = df['duration'].str.extract('(\d+)').astype(float)

In [13]:
# Country & Genre Insights
df['genre_count'] = df['listed_in'].str.split(',').apply(len)
df['primary_genre'] = df['listed_in'].str.split(',').str[0]

In [14]:
tableau_cols = [
    'show_id', 'type', 'title', 'director', 'country',
    'date_added', 'year_added', 'month_name', 'month_added',
    'release_year', 'rating', 'duration_mins', 'listed_in',
    'primary_genre', 'genre_count'
]

In [15]:
# Save the cleaned dataset for Tableau

df_tableau = df[tableau_cols]

df_tableau.to_csv('netflix_cleaned_data.csv', index=False)

print("✅ Tableau-ready dataset saved successfully")

✅ Tableau-ready dataset saved successfully
